In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from nltk.corpus import stopwords
import warnings
import re 
import string
warnings.filterwarnings("ignore")

In [59]:
maxTweets = 20000
# Creating list to append tweet data to
tweets_list2 = []
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Safaricom_Care since:2016-01-01 until:2022-12-15').get_items()):
    if i>maxTweets:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
tweets_df.to_csv("tweets.csv", index = False)

In [60]:
tweets_df['Text'] = tweets_df['Text'].str.lower() 

In [61]:
tweets_df.head()

,Datetime,Tweet Id,Text,Username
0,2022-12-14 23:47:00+00:00,1603174748451733504,ditto @safaricom_care https://t.co/sgk6l3yqhd,OnguruMeister
1,2022-12-14 23:41:18+00:00,1603173313052037120,"@ital_john hello john, the deadline was octobe...",Safaricom_Care
2,2022-12-14 23:35:29+00:00,1603171849428508673,@missy_k32 @abdulahiadan10 @safaricom_care @be...,I_im_kibagendi
3,2022-12-14 23:34:57+00:00,1603171716594692096,"@safaricom_care, @safaricomplc we demand you ...",MutonyeMutuku_
4,2022-12-14 23:19:59+00:00,1603167951682764800,"hey @safaricom_care @safaricomplc ,15th decemb...",ndungujosephn


In [62]:
tweets_df = tweets_df[tweets_df['Username'] != 'Safaricom_Care']

In [63]:
tweets_df.shape

(14786, 4)

In [64]:
tweets_df.head()

,Datetime,Tweet Id,Text,Username
0,2022-12-14 23:47:00+00:00,1603174748451733504,ditto @safaricom_care https://t.co/sgk6l3yqhd,OnguruMeister
2,2022-12-14 23:35:29+00:00,1603171849428508673,@missy_k32 @abdulahiadan10 @safaricom_care @be...,I_im_kibagendi
3,2022-12-14 23:34:57+00:00,1603171716594692096,"@safaricom_care, @safaricomplc we demand you ...",MutonyeMutuku_
4,2022-12-14 23:19:59+00:00,1603167951682764800,"hey @safaricom_care @safaricomplc ,15th decemb...",ndungujosephn
5,2022-12-14 23:00:12+00:00,1603162972456640517,@enockmogaka7 @safaricomplc @safaricom_care th...,cellestialtech


In [65]:
# Defining functions to clean data
def remove_links(tweet):
    """Takes in a string and removes weblinks from it"""
    tweet = re.sub(r"https\S+"," ",tweet)
    tweet = re.sub(r"bit.ly/\S+", " ",tweet)
    return tweet
def remove_user(tweet):
    """Takes in a string and removes @user information from it"""
    tweet = re.sub('(rt\s@[a-z]+[a-z0-9-_]+)', '',tweet)
    tweet = re.sub('(@[a-z]+[a-z0-9-_]+)', '',tweet)
    # tweet = re.sub("(sxsw)"," ",tweet)
    # tweet = re.sub("(link)"," ",tweet)
    return tweet
def remove_hasgtags(tweet):
    """Takes in a string and removes @user information from it"""
    tweeet = re.sub('(#[a-z]+[a-z0-9-_]+)', '',tweet)
    return tweet
def basic_clean(tweet):
    """Returns cleaned data, remove punctuation and numbers"""
    tweet = remove_user(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hasgtags(tweet)
    tweet = re.sub('['+ string.punctuation+ ']+', '',tweet) # Removes punctuation
    tweet = re.sub('([0-9]+)'," ",tweet) # Removes numbers
    return tweet


In [66]:
ls1=[]
for i in tweets_df["Text"]:
    ls1.append(basic_clean(i))

tweets_df['cleaned tweets']=ls1
    

In [67]:
tweets_df.head()

,Datetime,Tweet Id,Text,Username,cleaned tweets
0,2022-12-14 23:47:00+00:00,1603174748451733504,ditto @safaricom_care https://t.co/sgk6l3yqhd,OnguruMeister,ditto
2,2022-12-14 23:35:29+00:00,1603171849428508673,@missy_k32 @abdulahiadan10 @safaricom_care @be...,I_im_kibagendi,i can see you have started and clearly thi...
3,2022-12-14 23:34:57+00:00,1603171716594692096,"@safaricom_care, @safaricomplc we demand you ...",MutonyeMutuku_,we demand you issue a statement concerning ...
4,2022-12-14 23:19:59+00:00,1603167951682764800,"hey @safaricom_care @safaricomplc ,15th decemb...",ndungujosephn,hey th december is my birthday i can see it...
5,2022-12-14 23:00:12+00:00,1603162972456640517,@enockmogaka7 @safaricomplc @safaricom_care th...,cellestialtech,the man is dead you buggers still holding ...


In [68]:
df = tweets_df[['Datetime', 'cleaned tweets']]
df.head()

,Datetime,cleaned tweets
0,2022-12-14 23:47:00+00:00,ditto
2,2022-12-14 23:35:29+00:00,i can see you have started and clearly thi...
3,2022-12-14 23:34:57+00:00,we demand you issue a statement concerning ...
4,2022-12-14 23:19:59+00:00,hey th december is my birthday i can see it...
5,2022-12-14 23:00:12+00:00,the man is dead you buggers still holding ...


In [69]:
mpesa = {"pesa", "m-pesa", "esa", 'send money'}
airtime = {"credit", "card",'credo', 'credit', 'airtime'}
fibre = {'home fibre', 'faiba','safaricom home'}
def label_tweet(tweet):
    if mpesa.intersection(tweet.split(" ")):
        return "mpesa"
    elif airtime.intersection(tweet.split(" ")):
        return "credit"
    elif fibre.intersection(tweet.split(" ")):
        return "fibre"
    else:
        return "general"
# Add a new column to the dataframe with the labels for each tweet
df["LikeCount"] = df['cleaned tweets'].apply(label_tweet)
df["LikeCount"].value_counts()


general    14212
credit       388
mpesa        160
fibre         26
Name: LikeCount, dtype: int64